# Generate Infinite Tiktoks

## Generate video text
The first step to generate a tiktok, is obviously to know what you're going to stay.

To do so, my idea was to find something in genius.com, the famous website to have extra information on rap/pop songs. You can have informations about the context of the song when it was released.


In [78]:
import os
import requests
from moviepy.editor import *

In [79]:
def get_annotation(annotation_id):
    url = f"https://api.genius.com/annotations/{annotation_id}"
    payload = {}
    headers = {'Authorization': 'Bearer ' + os.environ['RAPGENIUS_API_KEY']}
    r = requests.request("GET", url, headers=headers, data=payload)
    data = r.json()
    
    full_annotation = ""
    parts = data['response']['annotation']['body']['dom']['children']

    full_annotation = find_annotation(parts)
    
    return full_annotation
def find_annotation(root):
    # print(type(root))
    full_annotation = ""
    if(type(root) == str):
        if(root == ""):
            root = '\n'
        full_annotation += root
    elif(type(root) == dict):
        if('children' in root):
            full_annotation += find_annotation(root['children'])
    elif(type(root) == list):
        for part in root:
            full_annotation += find_annotation(part)
    
    return full_annotation
def get_id_song(url):
    response = requests.get(url).content
    id = int(response.split(b'genius://songs/')[1].split(b'"')[0])
    return id
def get_song_description(song_id):
    url = f"https://api.genius.com/songs/{song_id}"

    payload = {}
    headers = {
        'Authorization': 'Bearer ' + os.environ['RAPGENIUS_API_KEY']
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    data = response.json()
    return get_annotation(data['response']['song']['description_annotation']['id'])


In [80]:
url = "https://genius.com/Eminem-houdini-lyrics"
video_name = "Eminem-Houdini"

song_description = get_song_description(get_id_song(url))
print(song_description)

“Houdini” is the lead single of Eminem’s twelfth studio album The Death of Slim Shady (Coup de Grâce). The track is named after Harry Houdini, a popular magician known for his death-defying stunts like the Chinese Water Torture Cell, which Eminem replicates in the intro to his 1999 track, “Role Model.”
With the return of long-time collaborator Jeff Bass of the Bass Brothers, Eminem revisits his catchy albeit morbid sound of the early 2000s. This fanfare track sees him callback to his old catchphrases such as Shady’s iconic “Guess who’s back?” line from the 2002 track “Without Me.” The song heavily samples and interpolates the Steve Miller Band’s 1982 track “Abracadabra.”
On October 24, 1926, the legendary Houdini tragically passed away in Detroit, Michigan at the age of 52. Interestingly, Detroit is also the hometown of Eminem, who turns 52 in October 2024.
The single was first teased by Eminem on May 20, 2024, when he posted a teaser to his social media accounts with a text message fr

## Create Audio for the video

In [81]:
from openai import OpenAI
import os

api_key = os.environ.get('OPENAI_API_KEY_LORRAIN')
client = OpenAI(api_key=api_key)

def text_to_speech_mp3(text, output_filename, model="tts-1", voice="alloy"):
    """
    Converts a text string to speech using the OpenAI API and writes the resulting audio to an MP3 file.

    Args:
    text (str): The text string to convert to speech.
    output_filename (str): The name of the output MP3 file.
    model (str): The model to use for speech synthesis. Defaults to 'tts-1'.
    voice (str): The voice to use for speech synthesis. Defaults to 'alloy'.
    """
    global client
    client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY_LORRAIN')) if client is None else client
    speech_file_path = output_filename


    with client.audio.speech.with_streaming_response.create(
        model=model,
        voice=voice,
        input=text,
    ) as response:
        response.stream_to_file(speech_file_path+('.mp3' if not speech_file_path.endswith('.mp3') else ''))

def getTextTimingsOfMp3(mp3file):
    global client
    client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY_LORRAIN')) if client is None else client
    
    audio_file = open(mp3file, "rb")
    transcript = client.audio.transcriptions.create(
        file=audio_file,
        model="whisper-1",
        response_format="verbose_json",
        timestamp_granularities=["word"]
    )

    return transcript


In [82]:
def onlychars(text):
    return ''.join(c if c.isalnum() or c.isspace() else '' for c in text)

def verify_transcription(original_text, transcription):
    """
    Verifies that the text is the same after the tts and stt.

    Args:
    original_text (str): The original text.
    transcription (dict): The transcription dictionary.

    Returns:
    int: ratio of the original text and the transcription text.
    """
    
    transcription_text = onlychars(transcription.text.lower())
    original_text = onlychars(original_text.lower())
    
    print('original :\n',original_text.replace('\n', ' '))
    print('transcription :\n',transcription_text,'\n')
    
    
    if transcription_text == original_text:
        return 1
    
    # correct_words = 0
    for word in original_text.split():
        # print(f'testing {word}')
        len_before = len(transcription_text)
        transcription_text = str(transcription_text).replace(word, '', 1)
        len_after = len(transcription_text)
        if(len_before == len_after):
            print(f'word {word} not found')
        # print(f'removed word ? {len_before - len_after}')
#
    print(len(original_text.split()), len(transcription_text.split()))
    return 1 - len(transcription_text.split()) / len(original_text.split())


In [83]:
text_to_speech_mp3(song_description, 'audio_description')
timing_description = getTextTimingsOfMp3('audio_description.mp3')

2024-07-03 16:33:01,107 - INFO - httpx - HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"
2024-07-03 16:33:19,637 - INFO - httpx - HTTP Request: POST https://api.openai.com/v1/audio/transcriptions "HTTP/1.1 200 OK"


In [84]:
print(timing_description)
print(verify_transcription(song_description, timing_description) * 100, "%")

onlychars_text = onlychars(timing_description.text)
final_text = timing_description.text

Transcription(text="Houdini is the lead single of Eminem's 12th studio album, The Death of Slim Shady, Coup De Grace. The track is named after Harry Houdini, a popular magician known for his death-defying stunts like the Chinese water torture cell, which Eminem replicates in the intro to his 1999 track, Role Model. With the return of longtime collaborator, Jeff Bass of the Bass Brothers, Eminem revisits his catchy albeit morbid sound of the early 2000s. This fanfare track sees him call back to his old catchphrases, such as Shady's iconic Guess Who's Back line from the 2002 track, Without Me. The song heavily samples and interpolates the Steve Miller band's 1982 track, Abracadabra. On October 24th, 1926, the legendary Houdini tragically passed away in Detroit, Michigan at the age of 52. Interestingly, Detroit is also the hometown of Eminem, who turns 52 in October 2024. The single was first teased by Eminem on May 20th, 2024, when he posted a teaser to his social media accounts with a t

## Find pictures

### Find picture descrition
The idea I came up with is to ask a LLM (mistral in my case) to find some nice moments in the description of the song

In [85]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

def get_description_pictures(story):
    api_key = os.environ["MISTRAL_API_KEY"]
    model = "open-mistral-7b"

    client = MistralClient(api_key=api_key)

    preprompt = """Your job will be to write the visual of a tiktok video: the images that show on screen. You have to find some nice moments in the story to place pictures that will show on screen. Tell me when you choose an image of what. In this format:
    DURING THE SENTENCE "[place the full sentence here]", SHOW AN IMAGE OF "[place the description of the image]".
    Answer only with the descriptions of all the images. 
    Try to understand the story and which words are important to show visually. for example if you want to show an album cover, try to give the name of the album and the artist. If you want to show a person, don't try to give the name of the person, but just the description of what they are doing because is very unlikely I'll be able to find this exact person doing this exact action.
    Please chose the most images as possible (dont be shy ;)) and if a sentence needs more than one image, you can split the sentence in two or more parts.
    But NEVER give two descriptions for the same sentence.  
    Try not to give a description, but key words because i will look online for the exact same words you describe. That means if you are too complex, i will not be able to find the image.
    Don't forget that the context will not be given to the website where I will find the images, that means if the description is too vague, I will not be able to find the image.
    Try to kep the descriptions short and concise.
    Before I give you an example, there is one last important thing : You need to choose a lot of moments, because the images will be shown for a short time, so you need to have a lot of images to show the whole story.
    
    ----------------
    For example:
    
    STORY : "Eonis, a wise and patient god, was intrigued by a Rubik's Cube he saw a human child playing with. He materialized one and began to solve it, but found it surprisingly challenging. Despite his divine knowledge, it took him months in the celestial realm to finally align the last color."
    
    DURING THE SENTENCE "Eonis, a wise and patient god", SHOW AN IMAGE OF "a god".
    DURING THE SENTENCE "was intrigued by a Rubik's Cube he saw a human child playing with", SHOW AN IMAGE OF "a rubik's cube".
    DURING THE SENTENCE "He materialized one and began to solve it", SHOW AN IMAGE OF "a person solving a rubik's cube".
    DURING THE SENTENCE "but found it surprisingly challenging", SHOW AN IMAGE OF "someone thinking very hard".
    DURING THE SENTENCE "Despite his divine knowledge, it took him months", SHOW AN IMAGE OF "a hourglass".
    DURING THE SENTENCE "in the celestial realm to finally align the last color", SHOW AN IMAGE OF "a solved rubiks cube".
    ----------------
    
    Now it is your turn:

    STORY : """

    messages = [
        ChatMessage(role="user", content=preprompt + f'"{story}"'),
    ]

    chat_response = client.chat(
        model=model,
        messages=messages,
    )

    results = chat_response.choices[0].message.content

    return results.split("\n\n")

def convert_to_dict(lines):
    visual_dict = {}
    for line in lines:
        print(line)
        if line:
            sentence, image_desc = line.split(', SHOW AN IMAGE OF ', 1)
            image_desc = image_desc.replace('"', '')
            sentence = sentence.split(' THE SENTENCE ')[1]
            sentence = str(sentence).replace('"', '', 2)
            visual_dict[sentence] = image_desc
    return visual_dict

def get_pictures(story):
    return convert_to_dict(get_description_pictures(story))


In [86]:
print(onlychars_text)
descriptions = get_pictures(onlychars_text)
print(descriptions)

Houdini is the lead single of Eminems 12th studio album The Death of Slim Shady Coup De Grace The track is named after Harry Houdini a popular magician known for his deathdefying stunts like the Chinese water torture cell which Eminem replicates in the intro to his 1999 track Role Model With the return of longtime collaborator Jeff Bass of the Bass Brothers Eminem revisits his catchy albeit morbid sound of the early 2000s This fanfare track sees him call back to his old catchphrases such as Shadys iconic Guess Whos Back line from the 2002 track Without Me The song heavily samples and interpolates the Steve Miller bands 1982 track Abracadabra On October 24th 1926 the legendary Houdini tragically passed away in Detroit Michigan at the age of 52 Interestingly Detroit is also the hometown of Eminem who turns 52 in October 2024 The single was first teased by Eminem on May 20th 2024 when he posted a teaser to his social media accounts with a text message from all contacts stating and for my 

2024-07-03 16:33:26,665 - INFO - httpx - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


DURING THE SENTENCE "Houdini is the lead single of Eminems 12th studio album The Death of Slim Shady Coup De Grace", SHOW AN IMAGE OF "Eminem album cover".
DURING THE SENTENCE "The track is named after Harry Houdini", SHOW AN IMAGE OF "Harry Houdini".
DURING THE SENTENCE "a popular magician known for his death-defying stunts like the Chinese water torture cell", SHOW AN IMAGE OF "Chinese water torture cell".
DURING THE SENTENCE "Eminem replicates in the intro to his 1999 track Role Model", SHOW AN IMAGE OF "Role Model intro".
DURING THE SENTENCE "With the return of longtime collaborator Jeff Bass of the Bass Brothers", SHOW AN IMAGE OF "Jeff Bass and the Bass Brothers".
DURING THE SENTENCE "Eminem revisits his catchy albeit morbid sound of the early 2000s", SHOW AN IMAGE OF "Eminem performing in the early 2000s".
DURING THE SENTENCE "This fanfare track sees him call back to his old catchphrases such as Shadys iconic Guess Whos Back line", SHOW AN IMAGE OF "Guess Whos Back lyrics".
DURI

### Generate or Find picture

In [100]:
from icrawler.builtin import GoogleImageCrawler

def remove_special_chars(text):
    return ''.join(e for e in text.replace(' ', '_') if e.isalnum() and not e == '_')

def dl_images(keywords, path, input_names = False, max_num = 2):
    for i, keyword in enumerate(keywords):
        try:
            print(f'Downloading images for {keyword}')
            current_path = os.path.join(path, (remove_special_chars(keyword) if input_names else str(i).zfill(3)))
            google_Crawler = GoogleImageCrawler(storage = {'root_dir': current_path})
            google_Crawler.crawl(keyword = keyword, max_num = max_num, file_idx_offset = 0)
        except Exception as e:
            print(f'Error while downloading images for {keyword} : error {e}')
# dl_images(['Houdini', 'Eminem'], 'images/tests', max_num=3)

In [101]:
import os
import shutil

def clear_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

clear_folder(f'images/{video_name}')

vals = list(descriptions.values())
dl_images(vals, f'images/{video_name}', max_num=5)

2024-07-03 16:44:53,714 - INFO - icrawler.crawler - start crawling...
2024-07-03 16:44:53,714 - INFO - icrawler.crawler - starting 1 feeder threads...
2024-07-03 16:44:53,716 - INFO - feeder - thread feeder-001 exit
2024-07-03 16:44:53,718 - INFO - icrawler.crawler - starting 1 parser threads...
2024-07-03 16:44:53,720 - INFO - icrawler.crawler - starting 1 downloader threads...


2024-07-03 16:44:54,488 - INFO - parser - parsing result page https://www.google.com/search?q=Eminem+album+cover.&ijn=0&start=0&tbs=&tbm=isch
2024-07-03 16:44:54,617 - INFO - downloader - image #1	https://m.media-amazon.com/images/I/71n0xmxpw7L._UF1000,1000_QL80_.jpg
2024-07-03 16:44:54,705 - INFO - downloader - image #2	https://cdns-images.dzcdn.net/images/cover/d6e14fe8e855c20db60b31a7b42eb007/1900x1900-000000-80-0-0.jpg
2024-07-03 16:44:55,167 - INFO - downloader - image #3	https://upload.wikimedia.org/wikipedia/en/8/80/Eminem_-_Music_to_Be_Murdered_By.png
2024-07-03 16:44:55,250 - INFO - downloader - image #4	https://i.redd.it/kzffnw0xzamb1.jpg
2024-07-03 16:44:55,323 - INFO - downloader - image #5	https://i.ebayimg.com/images/g/Jn4AAOSwfaxlRPv1/s-l1600.jpg
2024-07-03 16:44:55,432 - INFO - downloader - downloaded images reach max num, thread downloader-001 is ready to exit
2024-07-03 16:44:55,433 - INFO - downloader - thread downloader-001 exit
2024-07-03 16:44:55,723 - INFO - icra

2024-07-03 16:44:56,284 - INFO - parser - parsing result page https://www.google.com/search?q=Harry+Houdini.&ijn=0&start=0&tbs=&tbm=isch
2024-07-03 16:44:56,331 - INFO - parser - downloaded image reached max num, thread parser-001 is ready to exit
2024-07-03 16:44:56,340 - INFO - parser - thread parser-001 exit
2024-07-03 16:44:56,471 - INFO - downloader - image #1	https://medias.histoire-et-civilisations.com/api/v1/images/view/64f5e1aca134cf051010a6dd/width_1000/image.jpg
2024-07-03 16:44:57,111 - INFO - downloader - image #2	https://upload.wikimedia.org/wikipedia/commons/3/36/Harry_Houdini.png
2024-07-03 16:44:57,206 - ERROR - downloader - Response status code 404, file https://upload.wikimedia.org/wikipedia/commons/thumb/d/db/Harry_Houdini_jumps_from_Harvard_Bridge%2C_Boston%2C_Massachusetts%2C_1908_-_John_H._Thurston%2C_stereopticons_LCCN2015650990_-_restoration.jpg
2024-07-03 16:44:57,266 - INFO - downloader - image #3	https://static.wixstatic.com/media/fc6f03_c3261661618c46aa9293

2024-07-03 16:44:58,292 - INFO - parser - parsing result page https://www.google.com/search?q=Chinese+water+torture+cell.&ijn=0&start=0&tbs=&tbm=isch
2024-07-03 16:44:58,375 - INFO - parser - downloaded image reached max num, thread parser-001 is ready to exit
2024-07-03 16:44:58,376 - INFO - parser - thread parser-001 exit
2024-07-03 16:44:58,919 - INFO - downloader - image #1	https://upload.wikimedia.org/wikipedia/commons/e/e8/Houdini_performing_Water_Torture_Cell.jpg
2024-07-03 16:44:59,990 - INFO - downloader - image #2	http://www.geniimagazine.com/wiki/images/2/2e/WaterTortureCell.png
2024-07-03 16:45:00,409 - INFO - downloader - image #3	https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEhN_bm8otwoB78EB3jpiWJdheUPqpXaM1m1nKuJqh6ysTggFhGdsPU4264Wi0xVWxenzAKwGHgzxOWDhqa8L_US7cwKf57SlHzaZ8icrOtrkNNihc-xW34e85ggV1Pp5rprYQQF8GfW_F4/s1600/usd_chinese.jpg
2024-07-03 16:45:00,449 - INFO - downloader - image #4	https://c8.alamy.com/compfr/t9jx66/harry-houdini-la-torture-de-l-eau-c

2024-07-03 16:45:01,621 - INFO - parser - parsing result page https://www.google.com/search?q=Role+Model+intro.&ijn=0&start=0&tbs=&tbm=isch
2024-07-03 16:45:01,716 - INFO - downloader - image #1	https://image.slidesharecdn.com/rolemodel-150411091713-conversion-gate01/85/Role-model-2-320.jpg
2024-07-03 16:45:01,770 - INFO - downloader - image #2	https://i.ytimg.com/vi/55OJF945lc0/sddefault.jpg
2024-07-03 16:45:01,785 - INFO - downloader - image #3	https://i.ytimg.com/vi/XmY6nTH6n1A/hqdefault.jpg
2024-07-03 16:45:01,962 - INFO - downloader - image #4	https://images.slideplayer.com/26/8626487/slides/slide_3.jpg
2024-07-03 16:45:02,062 - INFO - downloader - image #5	https://miro.medium.com/v2/resize:fit:1100/1*EQxfu2Ieh7S9uZhWsX9EWA.png
2024-07-03 16:45:02,146 - INFO - downloader - downloaded images reach max num, thread downloader-001 is ready to exit
2024-07-03 16:45:02,147 - INFO - downloader - thread downloader-001 exit
2024-07-03 16:45:02,760 - INFO - icrawler.crawler - Crawling task 

...

2024-07-03 16:45:21,128 - INFO - parser - parsing result page https://www.google.com/search?q=Eminem+teaser+post.&ijn=0&start=0&tbs=&tbm=isch
Exception in thread parser-001:
Traceback (most recent call last):
  File "c:\Users\lorra\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\lorra\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\lorra\AppData\Local\Programs\Python\Python311\Lib\site-packages\icrawler\parser.py", line 94, in worker_exec
    for task in self.parse(response, **kwargs):
TypeError: 'NoneType' object is not iterable
2024-07-03 16:45:25,840 - INFO - downloader - no more download task for thread downloader-001
2024-07-03 16:45:25,843 - INFO - downloader - thread downloader-001 exit
2024-07-03 16:45:26,843 - INFO - icrawler.crawler - Crawling task done!
2024-07-03 16:45:26,845 - INFO - icrawler.crawler - start cr

2024-07-03 16:45:27,768 - INFO - parser - parsing result page https://www.google.com/search?q=Eminem%27s+text+message+teaser.&ijn=0&start=0&tbs=&tbm=isch
2024-07-03 16:45:27,926 - INFO - downloader - image #1	https://www.revolt.tv/article/media_1ca0ba8c46a73f84bccd2d0bb689f2039bc157512.jpeg?width=1200&format=pjpg
2024-07-03 16:45:27,986 - INFO - downloader - image #2	https://i.ytimg.com/vi/YrQgB1REkCs/oar2.jpg
2024-07-03 16:45:28,992 - INFO - downloader - image #3	https://eminem.news/wp-content/uploads/2020/05/my.community-681x1222.png
2024-07-03 16:45:29,067 - INFO - downloader - image #4	https://i.dailymail.co.uk/1s/2024/05/22/09/85189637-13446321-image-a-50_1716367850038.jpg
2024-07-03 16:45:29,141 - INFO - downloader - image #5	https://i.ytimg.com/vi/VNqOKsDN4Jo/maxresdefault.jpg
2024-07-03 16:45:29,184 - INFO - downloader - downloaded images reach max num, thread downloader-001 is ready to exit
2024-07-03 16:45:29,185 - INFO - downloader - thread downloader-001 exit
2024-07-03 16:

2024-07-03 16:45:30,728 - INFO - parser - parsing result page https://www.google.com/search?q=May+31st+date.&ijn=0&start=0&tbs=&tbm=isch
2024-07-03 16:45:30,972 - INFO - downloader - image #1	https://www.shutterstock.com/image-vector/may-31-vector-flat-daily-260nw-393566650.jpg
2024-07-03 16:45:31,224 - INFO - downloader - image #2	https://www.shutterstock.com/image-vector/may-31-calendar-icon-red-260nw-1039446784.jpg
2024-07-03 16:45:31,352 - ERROR - downloader - Response status code 400, file https://media.istockphoto.com/id/1135804836/photo/cubes-31st-may.jpg
2024-07-03 16:45:31,416 - INFO - downloader - image #3	https://static.vecteezy.com/system/resources/previews/022/109/406/non_2x/flat-icon-calendar-31-of-may-date-day-and-month-illustration-blue-teal-green-color-banner-31-may-31st-of-may-vector.jpg
2024-07-03 16:45:31,480 - INFO - downloader - image #4	https://s.wincalendar.net/img/en/days/may-31-2021.png
2024-07-03 16:45:31,584 - INFO - downloader - image #5	https://img.freepik

2024-07-03 16:45:32,524 - INFO - parser - parsing result page https://www.google.com/search?q=Eminem+and+David+Blaine+on+FaceTime.&ijn=0&start=0&tbs=&tbm=isch
2024-07-03 16:45:32,586 - INFO - parser - downloaded image reached max num, thread parser-001 is ready to exit
2024-07-03 16:45:32,586 - INFO - parser - thread parser-001 exit
2024-07-03 16:45:32,651 - INFO - downloader - image #1	https://pbs.twimg.com/ext_tw_video_thumb/1795632222634143744/pu/img/vfIjahSM8S_soya6.jpg
2024-07-03 16:45:32,704 - INFO - downloader - image #2	https://townsquare.media/site/812/files/2024/05/attachment-eminem-david-blaine1.jpg
2024-07-03 16:45:32,958 - INFO - downloader - image #3	https://img.buzzfeed.com/buzzfeed-static/static/2024-05/29/17/asset/a2b75ff62c80/sub-buzz-648-1717002725-3.png
2024-07-03 16:45:33,008 - INFO - downloader - image #4	https://i.ytimg.com/vi/QNbJoKUIw0I/sddefault.jpg
2024-07-03 16:45:33,352 - INFO - downloader - image #5	https://www.vibe.com/wp-content/uploads/2024/05/Em-Blaine

2024-07-03 16:45:34,356 - INFO - parser - downloaded image reached max num, thread parser-001 is ready to exit
2024-07-03 16:45:34,357 - INFO - parser - thread parser-001 exit
2024-07-03 16:45:34,861 - INFO - parser - parsing result page https://www.google.com/search?q=Eminem+and+David+Blaine+announcement+video.&ijn=0&start=0&tbs=&tbm=isch
2024-07-03 16:45:36,925 - INFO - downloader - image #1	https://img.buzzfeed.com/buzzfeed-static/static/2024-05/29/17/asset/a2b75ff62c80/sub-buzz-648-1717002725-3.png
2024-07-03 16:45:37,282 - INFO - downloader - image #2	https://www.vibe.com/wp-content/uploads/2024/05/Em-Blaine.png
2024-07-03 16:45:37,350 - INFO - downloader - image #3	https://townsquare.media/site/812/files/2024/05/attachment-eminem-david-blaine1.jpg
2024-07-03 16:45:37,736 - ERROR - downloader - Response status code 403, file https://content.invisioncic.com/y329496/monthly_2024_05/eminemdavidblanie.jpg
2024-07-03 16:45:37,776 - INFO - downloader - image #4	https://static.independen

## Create video from the pictures

In [102]:
from moviepy.editor import ImageClip, CompositeVideoClip, VideoFileClip, concatenate_videoclips

### Adding the pictures

In [103]:
import nltk
def find_closest_match(words, target):
    # print(f'try to find "{target}" in "{words}"')
    target_words = nltk.word_tokenize(target)
    similarities = []
    for i in range(len(words) - len(target_words) + 1):
        similarity = nltk.jaccard_distance(set(target_words), set(words[i:i+len(target_words)]))
        # print(f'similarity between {target_words} and {words[i:i+len(target_words)]} : {similarity}')
        similarities.append(similarity)
    # print(similarities)
    # print(type(similarities))
    best_match_index = similarities.index(min(similarities))

    return ' '.join(words[best_match_index:best_match_index+len(target_words)])


def get_sentence_timings(transcription, sentence):
    """
    Returns the start and end timings of a sentence in a transcription object.

    Args:
    transcription (Transcription): The transcription object.
    sentence (str): The sentence to search for.

    Returns:
    tuple: A tuple containing the start and end timings of the sentence.
    """
    word_list = [d['word'] for d in transcription.words]
    sentence = find_closest_match(word_list, sentence)
    # sentence = onlychars(sentence)
    # print(f'try to find {sentence} in {word_list}')
    
    words = transcription.words
    sentence = sentence.replace('...', ' ').replace('\"', ' ').replace('.', '')
    sentence_words = sentence.split()
    sentence_start = None
    sentence_end = None

    # print(f'try to find {sentence_words} in {words}')
    
    i = 0
    while i < len(words):
        if words[i]['word'].lower() == sentence_words[0].lower():
            # print(f'found {sentence_words[0]}')
            j = 1
            while j < len(sentence_words) and i + j < len(words) and words[i + j]['word'].lower() == sentence_words[j].lower():
                j += 1
            if j == len(sentence_words):
                sentence_start = words[i]['start']
                sentence_end = words[i + j - 1]['end']
                break
            # print(f'found {sentence_words[:j]} but not the rest')
        i += 1
        
    if(sentence_start == None or sentence_end == None):
        print(f"no timings found for '{sentence}'")
    else:
        print(f"timings found for '{sentence}' : {sentence_start} to {sentence_end}")    
        sentence_start = round(sentence_start, 2)
        sentence_end = round(sentence_end, 2)
    return sentence_start, sentence_end

In [104]:
def duplicate_video(input_video_path, output_video_path, duration = 0):
    clip = VideoFileClip(input_video_path)
    clip = clip.subclip(0, duration) if clip.duration > duration else clip
    clip.write_videofile(output_video_path)

In [105]:
if(not os.path.exists(f'videos/results/{video_name}.mp4')):
    duplicate_video('videos\satifying\satisfying.mp4', f'videos/results/{video_name}.mp4', timing_description.duration)

In [106]:
import os

def get_images_paths(directory):
    file_list = os.listdir(directory)

    file_list = [f for f in file_list if os.path.isfile(os.path.join(directory, f))]

    file_list.sort(key=lambda x: int(''.join(filter(str.isdigit, x))))

    file_paths = [os.path.join(directory, f) for f in file_list]

    return file_paths

In [107]:
import random

def select_images(root_dir, weight_decay=0.9):
    image_list = []
    for folder in os.listdir(root_dir):
        folder_path = os.path.join(root_dir, folder)
        if os.path.isdir(folder_path):
            images = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
            if images:
                weights = [weight_decay**i for i in range(len(images))]
                image_list.append(os.path.join(folder_path, images[random.choices(range(len(images)), weights=weights)[0]]))
    return image_list

In [108]:
i=0
timings = []
images = select_images(f'images/{video_name}')
m = 0
for key in list(descriptions.keys()):
    # timing_description is the original transcription
    # key is the sentence
    start_time, end_time = get_sentence_timings(timing_description, key)
    if(start_time < m):
        start_time = m
    m = end_time
    if(start_time == None or end_time == None):
        continue
    timings.append((start_time, end_time))
    # print(start_time, end_time)
    # add_image_to_video(f'videos/results/{video_name}.mp4', f'videos/results/{video_name}.mp4', f'images/{video_name}/{i}.jpg', start_time, end_time - start_time, 0, 0, 1280, 720)
    i+=1

print(timings)
print(images)
print(len(timings), len(images))

timings found for 'Houdini is the lead single of Eminem's 12th studio album The Death of Slim Shady Coup De Grace' : 0.0 to 6.28000020980835
timings found for 'The track is named after Harry Houdini' : 7.21999979019165 to 9.039999961853027
timings found for 'a popular magician known for his death defying stunts like the Chinese water torture' : 9.5600004196167 to 14.140000343322754
timings found for 'Eminem replicates in the intro to his 1999 track Role Model' : 15.279999732971191 to 19.280000686645508
timings found for 'With the return of longtime collaborator Jeff Bass of the Bass Brothers' : 20.1200008392334 to 23.739999771118164
timings found for 'Eminem revisits his catchy albeit morbid sound of the early 2000s' : 24.520000457763672 to 28.420000076293945
timings found for 'This fanfare track sees him call back to his old catchphrases such as Shady's iconic Guess Who's Back line' : 28.719999313354492 to 36.13999938964844
timings found for 'The song heavily samples and interpolates 

In [109]:
if(len(timings) != len(images)):
    print('Error')
    print("Number of images and timings do not match")
    

Error
Number of images and timings do not match


In [110]:
def add_multiple_images_to_video(input_video_path, output_video_path, images, timings):
    print(f"Adding {len(images)} images to the video")
    print(f"Images : {images}")
    print(f"Timings : {timings}")
    
    images = images.copy()
    clip = VideoFileClip(input_video_path, audio=False)
    clips = []
    offset = 0
    
    for start_time, end_time in timings:
        if start_time == None or end_time == None:
            print("Could not find timings")
            continue
        print(f"Adding image {images[0]} from {start_time} to {end_time}")
        # print(len(images))
        clip_before = clip.subclip(0, start_time-offset)
        clip_with_image = clip.subclip(start_time-offset, end_time-offset)
        clip_after = clip.subclip(end_time-offset, clip.duration)
        offset += clip_before.duration + clip_with_image.duration
        
        image_clip = ImageClip(images.pop(0))
        image_clip = image_clip.resize(width=clip.w/2, height=clip.h/2)
        image_clip = image_clip.set_pos(('center', 'center'))
        image_clip = image_clip.set_start(0).set_duration(clip_with_image.duration)

        final_clip_middle = CompositeVideoClip([clip_with_image, image_clip])

        final_clip = concatenate_videoclips([clip_before, final_clip_middle])

        clip = clip_after
        
        clips.append(final_clip)

    clips.append(clip)
    final_clip = concatenate_videoclips(clips, method="chain")

    # Save the video
    final_clip.write_videofile(output_video_path)

In [111]:
# video_name = "Eminem-Houdini"
add_multiple_images_to_video(f'videos/results/{video_name}.mp4', f'videos/results/{video_name}_V2.mp4', images, timings)

Adding 14 images to the video
Images : ['images/Eminem-Houdini\\000\\000004.jpg', 'images/Eminem-Houdini\\001\\000002.png', 'images/Eminem-Houdini\\002\\000003.jpg', 'images/Eminem-Houdini\\003\\000002.jpg', 'images/Eminem-Houdini\\004\\000005.jpg', 'images/Eminem-Houdini\\005\\000005.jpg', 'images/Eminem-Houdini\\006\\000005.jpg', 'images/Eminem-Houdini\\007\\000003.jpg', 'images/Eminem-Houdini\\008\\000004.jpg', 'images/Eminem-Houdini\\009\\000002.jpg', 'images/Eminem-Houdini\\011\\000004.jpg', 'images/Eminem-Houdini\\012\\000004.png', 'images/Eminem-Houdini\\013\\000004.jpg', 'images/Eminem-Houdini\\014\\000003.jpg']
Timings : [(0.0, 6.28), (7.22, 9.04), (9.56, 14.14), (15.28, 19.28), (20.12, 23.74), (24.52, 28.42), (28.72, 36.14), (38.78, 44.78), (45.34, 52.82), (53.56, 59.44), (59.44, 63.22), (64.18, 71.08), (71.82, 77.7), (77.82, 85.56), (85.56, 99.44)]
Adding image images/Eminem-Houdini\000\000004.jpg from 0.0 to 6.28
Adding image images/Eminem-Houdini\001\000002.png from 7.22 t

IndexError: list index out of range

## Sounds

### The voice

In [ ]:
# imports
from moviepy.editor import AudioFileClip, CompositeAudioClip

In [ ]:
def add_audio_to_video(video_path, audio_path, output_path):
    # Load the video and the audio file
    video = VideoFileClip(video_path)
    audio = AudioFileClip(audio_path)

    # Make the audio file as long as the video file
    final_audio = CompositeAudioClip([audio.set_duration(video.duration)])

    # Set the audio of the video to the final_audio
    final_video = video.set_audio(final_audio)

    # Write the result to a file
    final_video.write_videofile(output_path)

In [ ]:
add_audio_to_video(f'videos/results/{video_name}_V2.mp4', 'audio_description.mp3', f'videos/results/{video_name}_V3.mp4')

Moviepy - Building video videos/results/Eminem-Houdini_V3.mp4.
MoviePy - Writing audio in Eminem-Houdini_V3TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video videos/results/Eminem-Houdini_V3.mp4



Moviepy - Done !
Moviepy - video ready videos/results/Eminem-Houdini_V3.mp4


### Music

## Subtitles

In [ ]:
from moviepy.editor import *

def add_subtitles_to_video(video_path, subtitles, output_path, position = ('center', 0.1)):
    # Load the video
    print(video_path)
    video = VideoFileClip(video_path)

    # Create a list of TextClip objects for each subtitle
    text_clips = []
    for word in subtitles.words:
        # print(word['word'], word['start'], word['end'])
        text_clip = TextClip(word['word'], fontsize=60, color='white')
        text_clip = text_clip.set_start(word['start']).set_duration(word['end'] - word['start'])
        text_clip = text_clip.set_position(position)
        text_clips.append(text_clip)

    # Overlay the text clips on the video
    final_video = CompositeVideoClip([video] + text_clips)
    
    final_video.write_videofile(output_path)

In [ ]:
add_subtitles_to_video(f'videos/results/{video_name}_V3.mp4', timing_description, f'videos/results/{video_name}_V4.mp4')

videos/results/Eminem-Houdini_V3.mp4
Moviepy - Building video videos/results/Eminem-Houdini_V4.mp4.
MoviePy - Writing audio in Eminem-Houdini_V4TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video videos/results/Eminem-Houdini_V4.mp4



t: 100%|█████████▉| 3020/3024 [00:47<00:00, 79.46it/s, now=None]2024-07-03 16:23:30,551 - WARNING - py.warnings - c:\Users\lorra\AppData\Local\Programs\Python\Python311\Lib\site-packages\moviepy\video\io\ffmpeg_reader.py:123: UserWarning: Warning: in file videos/results/Eminem-Houdini_V3.mp4, 2764800 bytes wanted but 0 bytes read,at frame 3022/3024, at time 100.73/100.78 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

2024-07-03 16:23:30,564 - WARNING - py.warnings - c:\Users\lorra\AppData\Local\Programs\Python\Python311\Lib\site-packages\moviepy\video\io\ffmpeg_reader.py:123: UserWarning: Warning: in file videos/results/Eminem-Houdini_V3.mp4, 2764800 bytes wanted but 0 bytes read,at frame 3023/3024, at time 100.77/100.78 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready videos/results/Eminem-Houdini_V4.mp4
